In [ ]:
import numpy as np
import open3d as o3d

In [3]:
from typing import Annotated, Literal, TypeVar, Optional, Tuple
import numpy.typing as npt

DType = TypeVar("DType", bound=np.generic)
Array3 = Annotated[npt.NDArray[DType], Literal[3]]
ArrayNx3 = Annotated[npt.NDArray[DType], Literal["N", 3]]
ArrayNx4 = Annotated[npt.NDArray[DType], Literal["N", 4]]
ArrayNx8 = Annotated[npt.NDArray[DType], Literal["N", 8]]

In [16]:
class Plane:
    """
    Class represents plane, which contains relevant properties.
    """
    def __init__(self) -> None:
        self.points: ArrayNx3[np.float64] = None
        self.residual_value: Optional[np.float64] = None
        self.normal_vector: Optional[ArrayNx4[np.float64]] = None


    def insert(self, point: Array3[np.float64]) -> None:
        if self.points is None:
            self.points = np.array([point])
            return
        
        self.points = np.append(self.points, [point], axis=0)
        self.residual_value = None
        self.normal_vector = None


    def calculate_residual_value(self) -> Optional[np.float64]:
        if self.points is None:
            return None

        self.calculate_normal_vector()
        self.residual_value = np.abs(self.points @ self.normal_vector) / np.linalg.norm(self.normal_vector)

        return self.residual_value


    def calculate_normal_vector(self) -> Optional[ArrayNx4[np.float64]]:
        if self.points is None:
            return None
        
        o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(self.points))
        self.normal_vector, _ = o3d_pcd.segment_plane(
            distance_threshold=0.1, ransac_n=3, num_iterations=5000
        )

        return self.normal_vector


class Octree:
    """
    Class represents octree struct to segment planes from it.
    """
    def __init__(self, depth: int) -> None:
        assert depth >= 0
        
        self.depth: int = depth
        self.is_empty: bool = True
        self.children: Optional[ArrayNx8] = [None] * 8
        self.plane: Optional[Plane] = None
        self.bottom_bound: Optional[int] = None
        self.top_bound: Optional[int] = None


    def get_point_cloud_bounds(self, point_cloud: o3d.geometry.PointCloud) -> Tuple[np.float64, np.float64]:
        if self.bottom_bound is None and self.top_bound is None:
            o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(point_cloud.points))
            self.top_bound, self.bottom_bound = o3d_pcd.get_min_bound(), o3d_pcd.get_max_bound()

        return self.top_bound, self.bottom_bound
    

    def get_octet_number(self, point: Array3[np.float64]) -> int:
        center_x, center_y, center_z = (self.top_bound + self.bottom_bound) / 2
        point_x, point_y, point_z = point

        if point_x < center_x:
            if point_y < center_y:
                if point_z < center_z:
                    return 0
                else:
                    return 4
            else:
                if point_z < center_z:
                    return 1
                else:
                    return 5
        else:
            if point_y < center_y:
                if point_z < center_z:
                    return 3
                else:
                    return 7
            else:
                if point_z < center_z:
                    return 2
                else:
                    return 6
                

    def insert(self, point: Array3[np.float64]) -> None:
        if self.depth == 0:
            self.plane = Plane()
            self.plane.insert(point)
            return

        octet = self.get_octet_number(point)

        if self.children[octet] is None:
            self.children[octet] = Octree(self.depth - 1)

        self.children[octet].insert(point)


    def build(self, point_cloud: o3d.geometry.PointCloud):
        assert len(point_cloud.points) != 0

        self.get_point_cloud_bounds(point_cloud)
        self.is_empty = False

        for point in point_cloud.points:
            self.insert(point)

    
    def visualize(self) -> None:
        pass

<img src="assets/voxel.jpg" width="900" height="640" />

In [5]:
pcd_file = "../data/data/clouds/0.pcd"
point_cloud = o3d.io.read_point_cloud(pcd_file)
point_cloud_points = point_cloud.points

std::vector<Eigen::Vector3d> with 22096 elements.
Use numpy.asarray() to access data.

In [17]:
octree = Octree()